<a href="https://colab.research.google.com/github/soave424/pythondata/blob/main/%EB%B3%91%EC%95%84%EB%A6%AC%EB%B0%985%EC%A3%BC%EC%B0%A8%EB%AF%B8%EC%85%98(%EC%A3%A0%EC%9D%B4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개인프로젝트
- 1인 1역 정하기 
1. 1인 1역 이름 / 내용  입력 (csv)
2. 학생 명단 (csv) 
3. 역할별 학생 수 정하기 
4. 학생 배정 (랜덤) 
5. 순서대로 다음 역할 나오는 옵션 / 랜덤 옵션
6. 랜덤으로 했을 때 몇 번씩 했는지 그래프 
7. 만약 3번 이상 반복이면 다른 역할 배정 

In [ ]:
!apt-get install fonts-nanum -qq > /dev/null
!fc-cache -fv
import matplotlib as mpl
plt.rc('font', family = 'NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
mpl.font_manager._rebuild()
findfont = mpl.font_manager.fontManager.findfont
mpl.font_manager.findfont = findfont
mpl.backends.backend_agg.findfont = findfont

In [1]:
# 1인 1역과 학생 명단 파일 올리기 
from google.colab import files
files.upload()

Saving student.csv to student.csv


{'student.csv': b'\xec\xb6\x9c\xec\x84\x9d\xeb\xb2\x88\xed\x98\xb8,\xec\x9d\xb4\xeb\xa6\x84,,,,,\r\n1,\xea\xb0\x95\xec\x8a\xb9\xea\xb8\xb0,,,,,\r\n2,\xea\xb9\x80\xec\x9d\xb4\xec\xa4\x80,,,,,\r\n3,\xea\xb9\x80\xec\xa7\x84\xed\x9c\x98,,,,,\r\n4,\xeb\xb0\x95\xec\x83\x81\xec\xa7\x84,,,,,\r\n5,\xec\x84\x9c\xed\x83\x9c\xec\x9b\x85,,,,,\r\n6,\xec\x8b\xa0\xec\x8a\xb9\xeb\xaf\xbc,,,,,\r\n7,\xec\x98\xa4\xec\x9c\xa4\xed\x98\xb8,,,,,\r\n8,\xec\x9c\xa4\xeb\xaf\xbc\xec\xa4\x80,,,,,\r\n9,\xec\x9d\xb4\xec\xa4\x80\xed\x98\x81,,,,,\r\n10,\xed\x99\x8d\xec\x9c\xa4\xea\xb8\xb0,,,,,\r\n11,\xed\x99\x8d\xec\xa4\x80\xea\xb8\xb0,,,,,\r\n12,\xed\x99\xa9\xec\x9c\xa0\xec\xb0\xac,,,,,\r\n21,\xea\xb0\x95\xeb\xaf\xbc\xec\x84\x9c,,,,,\r\n22,\xea\xb9\x80\xeb\xaf\xbc\xec\xa0\x95,,,,,\r\n23,\xea\xb9\x80\xec\x84\x9c\xec\xa7\x84,,,,,\r\n24,\xea\xb9\x80\xec\x86\x8c\xec\x9d\x80,,,,,\r\n25,\xea\xb9\x80\xec\x98\x88\xec\x8a\xac,,,,,\r\n27,\xec\x9b\x90\xec\x98\x88\xeb\xb4\x84,,,,,\r\n28,\xec\x9d\xb4\xec\x8b\x9c\xec\x98\xa8,,,,,\

In [15]:
# CSV 파일 리스트로 만들기 (1인 1역, 학생명단) 

import csv
data_s = csv.reader(open('student.csv'))
data_r = csv.reader(open('roll.csv'))
next(data_r)
next(data_s)
rollall = []
rollper = {}
rolllist={}
student = []
for row in data_r : 
  rolllist[row[0]]=row[1]
  rollper[row[0]]=int(row[2])
  for i in range(int(row[2])):
    rollall.append(row[0])
for name in data_s :
  student.append(name[1])

In [16]:
# 학생명단 리스트와 1인 1역 리스트 일대일 대응 (순서대로)
match = {K:V for K, V in zip(student, rollall)}
print(match)

{'강승기': '급식', '김이준': '급식', '김진휘': '급식', '박상진': '급식', '서태웅': '급식', '신승민': '검사', '오윤호': '교실 앞 쓸기', '윤민준': '교실 뒤 쓸기', '이준혁': '칠판', '홍윤기': '칠판', '홍준기': '복도 쓸기', '황유찬': '복도 쓸기', '강민서': '1분단 쓸기', '김민정': '2분단 쓸기', '김서진': '3분단 쓸기', '김소은': '창틀', '김예슬': '창틀', '원예봄': '신발장', '이시온': '신발장', '이준이': '책장', '장하린': '쓰레기통 비우기', '정유리': '쓰레기통 비우기', '정하윤': '분리수거', '하예린': '분리수거'}


In [ ]:
# 학생명단 순서를 랜덤으로 섞어서 일대일 대응
import random 

In [ ]:
# 학생명단 순서에서 하나씩 이동하여 일대일 대응

In [ ]:
# 학생별 리스트를 만들어서 맡은 역할을 누적해가기  

In [ ]:
# 누적 데이터를 가지고 그래프 만들어보기 

In [ ]:
# (계속 랜덤이 경우에만) 역할이 뽑혔을 때 3번 했으면 다른 역할로 정하기 
# 순서대로 했을 경우 이미 다른 학생들이 역할이 다 배정되어 있는 경우면 어떡하지? 
# 3번인 경우 먼저 찾아서 그 경우 학생을 먼저 배정하고?  